In [1]:
import geopandas as gpd
import pandas as pd

In [3]:
conversion_table = pd.read_csv('/datadrive/parcel_id_to_pin_conversion_table.csv')

In [4]:
conversion_table.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9269565 entries, 0 to 9269564
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   parcel_id  9269565 non-null  object
 1   pin        9269565 non-null  object
dtypes: object(2)
memory usage: 141.4+ MB


In [ ]:
gis_data = gpd.read_file('/datadrive/florida_2023.geojson')

In [ ]:
gis_data.info(show_counts=True)

In [ ]:
gis_data = gis_data.merge(conversion_table, left_on='PARCELNO', right_on='parcel_id')

In [ ]:
gis_data.info(show_counts=True)
gis_data.to_file('/datadrive/florida_2023_with_pin.geojson', driver='GeoJSON') 

In [2]:
gis_data = gpd.read_file('/datadrive/florida_2023_with_pin.geojson')
latest_sales_data = pd.read_csv('/datadrive/latest_sales_data.csv')

/tmp/ipykernel_30634/486010570.py:2: DtypeWarning: Columns (0,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  latest_sales_data = pd.read_csv('/datadrive/latest_sales_data.csv')


In [5]:
latest_sales_data.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3779844 entries, 0 to 3779843
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   ParcelID    525986 non-null   object 
 1   Sale Date   3744561 non-null  object 
 2   Sale Price  3779843 non-null  object 
 3   PIN         3246419 non-null  object 
 4   Alt Key     7297 non-null     float64
dtypes: float64(1), object(4)
memory usage: 144.2+ MB


In [6]:
# First, merge on 'PIN' and 'pin'
merged_data = gis_data.merge(latest_sales_data, left_on='pin', right_on='PIN', how='left')

merged_data.info(show_counts=True)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 11178937 entries, 0 to 11178936
Data columns (total 9 columns):
 #   Column      Non-Null Count     Dtype   
---  ------      --------------     -----   
 0   PARCELNO    11178937 non-null  object  
 1   parcel_id   11178937 non-null  object  
 2   pin         11178937 non-null  object  
 3   geometry    10826206 non-null  geometry
 4   ParcelID    0 non-null         object  
 5   Sale Date   2417230 non-null   object  
 6   Sale Price  2417230 non-null   object  
 7   PIN         2417230 non-null   object  
 8   Alt Key     0 non-null         float64 
dtypes: float64(1), geometry(1), object(7)
memory usage: 767.6+ MB


In [7]:
# Then, for the rows that didn't find a match, merge on 'ParcelID' and 'parcel_id'
no_pin_match = merged_data[merged_data['PIN'].isna()]
match_on_parcel_id = no_pin_match.merge(latest_sales_data, left_on='parcel_id', right_on='ParcelID', how='left')

match_on_parcel_id.info(show_counts=True)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 8828384 entries, 0 to 8828383
Data columns (total 14 columns):
 #   Column        Non-Null Count    Dtype   
---  ------        --------------    -----   
 0   PARCELNO      8828384 non-null  object  
 1   parcel_id     8828384 non-null  object  
 2   pin           8828384 non-null  object  
 3   geometry      8470914 non-null  geometry
 4   ParcelID_x    0 non-null        object  
 5   Sale Date_x   0 non-null        object  
 6   Sale Price_x  0 non-null        object  
 7   PIN_x         0 non-null        object  
 8   Alt Key_x     0 non-null        float64 
 9   ParcelID_y    340627 non-null   object  
 10  Sale Date_y   309795 non-null   object  
 11  Sale Price_y  340626 non-null   object  
 12  PIN_y         0 non-null        object  
 13  Alt Key_y     0 non-null        float64 
dtypes: float64(2), geometry(1), object(11)
memory usage: 943.0+ MB


In [8]:
# Combine the two dataframes
final_merged_data = pd.concat([merged_data[~merged_data['PIN'].isna()], match_on_parcel_id])
final_merged_data.info(show_counts=True)

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 11245614 entries, 2053640 to 8828383
Data columns (total 19 columns):
 #   Column        Non-Null Count     Dtype   
---  ------        --------------     -----   
 0   PARCELNO      11245614 non-null  object  
 1   parcel_id     11245614 non-null  object  
 2   pin           11245614 non-null  object  
 3   geometry      10888137 non-null  geometry
 4   ParcelID      0 non-null         object  
 5   Sale Date     2417230 non-null   object  
 6   Sale Price    2417230 non-null   object  
 7   PIN           2417230 non-null   object  
 8   Alt Key       0 non-null         float64 
 9   ParcelID_x    0 non-null         object  
 10  Sale Date_x   0 non-null         object  
 11  Sale Price_x  0 non-null         object  
 12  PIN_x         0 non-null         object  
 13  Alt Key_x     0 non-null         float64 
 14  ParcelID_y    340627 non-null    object  
 15  Sale Date_y   309795 non-null    object  
 16  Sale Price_y  340626 non-n

In [9]:
final_merged_data.drop(columns=['ParcelID', 'ParcelID_x', 'Sale Date_x', 'Sale Price_x', 'PIN_x', 'PIN_y'], inplace=True)
final_merged_data['Sale Price'] = final_merged_data['Sale Price'].fillna(final_merged_data['Sale Price_y'])
final_merged_data['Sale Date'] = final_merged_data['Sale Date'].fillna(final_merged_data['Sale Date_y'])
final_merged_data = final_merged_data.drop_duplicates(subset=['parcel_id', 'pin', 'Sale Date', 'Sale Price'], keep='first')
final_merged_data.dropna(subset=['Sale Price'], inplace=True)
final_merged_data = final_merged_data[['parcel_id', 'pin', 'Sale Date', 'Sale Price', 'geometry']]
final_merged_data.reset_index(drop=True, inplace=True)
final_merged_data.to_file('/datadrive/final_sales_gis_file.geojson', driver='GeoJSON')

In [10]:
final_merged_data.info(show_counts=True)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2676886 entries, 0 to 2676885
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype   
---  ------      --------------    -----   
 0   parcel_id   2676886 non-null  object  
 1   pin         2676886 non-null  object  
 2   Sale Date   2653235 non-null  object  
 3   Sale Price  2676886 non-null  object  
 4   geometry    2643537 non-null  geometry
dtypes: geometry(1), object(4)
memory usage: 102.1+ MB
